In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.neighbors import KNeighborsClassifier

In [37]:
%%time
db_full = pd.read_csv('02-14-2018.csv', low_memory=False)

CPU times: user 11 s, sys: 2.31 s, total: 13.3 s
Wall time: 13.5 s


In [57]:
def preparing_data(db_full):
    """Функция кодирует метки целевого признака. Отбирает признаки методом случайного леса. 
    Возвращает готовые для обучения моделей данные"""    
    #кодирование меток
    
    categories = {}
    for key, value in enumerate(db_full['Label'].unique()):
        categories[value] = key + 1
    db_full['Category_Label'] = db_full['Label'].map(categories)
    
    categories = {}
    for key, value in enumerate(db_full['Dst Port'].unique()):
        categories[value] = key + 1
    db_full['Dst Port'] = db_full['Dst Port'].map(categories)
    
    #отбор признаков с помощью модели случайного леса

    X = db_full.select_dtypes(exclude=['object']).copy()
    X.drop(columns=['Flow Byts/s', 'Flow Pkts/s', 'Category_Label'],
           axis = 1, inplace=True)
    y = db_full['Category_Label']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=6)
    X_train.shape, y_train.shape

    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    
    importnace_features = pd.Series(clf.feature_importances_, X.columns)#.sort_values(ascending = False)
    importnace_features = importnace_features.head(n = 8).index
    
    X = db_full[importnace_features]
    y = db_full['Category_Label']
    
    types_at = db_full['Label'].unique()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=6)
    
    return importnace_features, types_at

In [58]:
importnace_features, types_at = preparing_data(db_full)

In [44]:
importnace_features

Index(['Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max'],
      dtype='object')

In [2]:
db = pd.read_csv('02-20-2018.csv')

In [3]:
db.columns


Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE 